第5回 モデルのトレーニング（最適化）

今回はモデルの学習について見ていきます．本当は一気にトレーニング関数の説明まで行きたいところですが，まずは学習の原理について触れていきます．

1，恐れるな自動微分

まあ，嫌な節のタイトルですね．なんせいきなり「微分」ですもんね．それに自動って...　しかし，この自動微分こそがpytorchのモデル学習の根幹と言えます．まずは微分とは何かという事を簡単に理解していきましょう．

微分とは，ある関数の微小区間での変化の割合を求める計算のことです．変化の割合は次のような式で求める事ができます．

In [1]:
y=lambda x: 2*x**2 #y=2x^2　を示す　lambdaというのは無名関数を作るのに使えます．
x1=1
x2=2

dif=(y(x2)-y(x1))/(x2-x1) #difに変化の割合を代入

print(dif)

6.0


これは，y=2x^2という関数があったとき，x が x1 から x2 へ変化した時の y の変化量を示しています．この場合，変化の区間は[x1，x2]であり，その大きさは x2-x1 で示せます．

ここで区間とは x が変化する幅である事がわかりましたね？ではこの区間を「めちゃくちゃ小さく」してみましょう．

In [2]:
y=lambda x: 2*x**2 #y=2x^2　を示す　lambdaというのは無名関数を作るのに使えます．
x1=1
x2=1+0.00000001 #x1からの差は0.00000001

dif=(y(x2)-y(x1))/(x2-x1) #difに変化の割合を代入

print(dif)

4.0


この滅茶苦茶小さくした区間のことを微小区間といい，微小区間での変化の割合を微分係数をといいます．そして，この微分係数を求めることを微分（数値微分）といいます．

つまり，微分とは超ちっちゃい区間での関数の変化の割合を求めるということです．しかし，これがいったい何の役に立つのでしょうか？それは，変数をどの向き（+ or -）に動かしたら関数の値が小さくなるのかを求めるためです．

値の変化を考える関数を「損失関数」として考えてみましょう．損失関数の値は常に小さくなる方がいいですよね？そのため，２億個以上存在するパラメータ（重みやバイアス）の値をそれぞれ増やすか減らすかによって損失関数の値が増えるか減るかを求める必要があります．

つまり，損失関数の値が小さくなる「方向」（どのパラメータの値を＋するか，-するか）を定める必要があるわけです．

この計算をpytorchでは自動で行ってくれます．変数の種類が2つ以上の微分を偏微分といい，pytorchでは2億以上のパラメータでの偏微分を行いますが... ここからは3年後期の内容なので今は蓋をしておきます．

2， 損失関数

いよいよトレーニング関数までの道のりを歩いていきます．その前に下準備をしておきましょう．次のコードを実行してください．

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

learning_rate = 1e-3
batch_size = 64
epochs = 5

まずは損失関数からです．損失関数は，モデルの出力と正解の値（ラベル値）との誤差を求める関数でしたね．この損失関数にはいろんな種類があります．それぞれに適した使い道が存在するので代表的なのを見ておきましょう．

参考URL https://qiita.com/lucasta1/items/3e0e4306940fc35e0af1

CrossEntropyLoss...多値分類（出力が3つ以上）でも2値分類でも使える優秀な損失関数です．迷ったらこれ使え！ってやつです．理由としては最終出力の関数
「SoftMax関数」との相性がとても良いからです．

さすがに詳しい定義式とかは省きます．難しすぎるのでね．代わりに呼び出し方を記しておきます．

In [4]:
from torch import nn

loss_fn=nn.CrossEntropyLoss() #損失関数呼び出し

input = torch.randn(5) #5つランダムな値
softmax=nn.Softmax() #ソフトマックス層
relu=nn.ReLU() #ReLU層
input=relu(input) #ReLU層にデータデータを入力
output=softmax(input) #ソフトマックスにデータ入力

target = torch.zeros(5) #ラベルデータ
target[2]=1 #正解ラベル2

loss = loss_fn(output, target) #損失を出す

print(output) #入力を表示
print(target) #出力を表示
print("loss is",loss) #損失の値を表示

tensor([0.5565, 0.1124, 0.1494, 0.1145, 0.0672])
tensor([0., 0., 1., 0., 0.])
loss is tensor(1.6776)


C:\Users\user\AppData\Local\Temp\ipykernel_13112\3846959998.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output=softmax(input) #ソフトマックスにデータ入力


Multi Margin Loss...複数クラス分類（多値分類）に使われる関数．

In [5]:
loss_fn = torch.nn.MultiMarginLoss() #マルチマージンロスの呼び出し

x1 = torch.tensor([[0.0, 0.0, 0.0, 1.0]])
x2 = torch.tensor([[0.1, 0.2, 0.4, 0.8]])
y = torch.tensor([3]) #3が正解ラベル

loss = loss_fn(x1, y) #正解時の損失の表示
loss2 = loss_fn(x2, y) #正解に近いときの損失
print(f"x1のloss={loss} , x2のloss={loss2}")

x1のloss=0.0 , x2のloss=0.32499998807907104


Kullback Leibler divergence Loss...KLダイバージェンスと呼ばれるもの．距離関数と呼ばれるものの一つ．

このKLダイバージェンスは理解がとても難しいです．ですが，分かりやすかったサイトがあるので示しておきます．
https://dx-consultant-fast-evolving.com/the-meaning-of-kl-divergence/

KLダイバージェンスは主に生成系のモデルというものに使われます．例えばオートエンコーダとかですね．また，皆さんが聞いたことがあるものとして「novelAI」がありますが，これはまた別のダイバージェンス関数が使われています．

In [6]:
loss_fn = nn.KLDivLoss() #KLダイバージェンス呼び出し

#ランダムなデータをソフトマックスにかけて，0~1にする．
_input = torch.randn(3, requires_grad=True) 
input = nn.Softmax()(_input)
_target = torch.randn(3, requires_grad=True)
target = nn.Softmax()(_target)

loss = loss_fn(input, target) #損失の算出
print(input, target)
print(loss)

tensor([0.2387, 0.2010, 0.5603], grad_fn=<SoftmaxBackward0>) tensor([0.4609, 0.1143, 0.4249], grad_fn=<SoftmaxBackward0>)
tensor(-0.4465, grad_fn=<MeanBackward0>)


C:\Users\user\AppData\Local\Temp\ipykernel_13112\994188236.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = nn.Softmax()(_input)
C:\Users\user\AppData\Local\Temp\ipykernel_13112\994188236.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  target = nn.Softmax()(_target)
c:\Users\user\venv\torch\lib\site-packages\torch\nn\functional.py:2916: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


いろんな損失関数がありますね．今回はこの中でもCrossEntropyLossを使います．

In [7]:
loss_fn=nn.CrossEntropyLoss()

3，オプティマイザ

次にオプティマイザを用意します．オプティマイザには損失関数の出力をもとにパラメータを調整するという役割があります．こちらにも種類がありますが基本的には最新のものを使うのがベストでしょう．

参考　https://rightcode.co.jp/blog/information-technology/torch-optim-optimizer-compare-and-verify-update-process-and-performance-of-optimization-methods

最新のものとしてはAdamWやAdamがありますが．その中でも一番シンプルなSGDを今回は使います．

オプティマイザは主にモデルのパラメータと，学習率の二つを引数に取ります．

In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

学習率は，パラメータの値をどれほど変更させるかを示す値です．これが大きいと学習が早く進みますが，精度が低くなります．

In [10]:
#一応AdamとAdamWの呼び出し方も書いときます
torch.optim.Adam
torch.optim.AdamW

torch.optim.adamw.AdamW

4，最適化を始める

それではモデルの学習，つまり最適化を始めていきましょう．まずは，トレーニング関数の定義です．

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):

    #enumerateは要素のindexと要素自体を渡す組み込み関数
    for batch, (X, y) in enumerate(dataloader):

        #モデルの出力を代入
        pred = model(X)

        #ラベルと出力を損失関数に代入し，損失を求める
        loss = loss_fn(pred, y)

        #勾配の計算を初期化
        optimizer.zero_grad()

        #ここで，どの方向に各パラメータを変化させることで
        #損失が小さくなるか求めている．
        loss.backward()

        #最適化を進める
        optimizer.step()

        if batch%100==0:
            print(f"loss {loss:>7f}")

これで，学習自体はできるようになりましたが，性能検証用の関数も作っておきましょう．

In [10]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset) #データの数
    num_batches = len(dataloader) #バッチサイズ
    test_loss, correct = 0, 0

    #ここでは微分の計算をしないのでwith torch.no_grad()を用いる
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches #損失の平均
    correct /= size #モデルの正確さの平均を示す．
    print(f"Test Loss: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
epochs = 10 #学習回数は10回
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss 2.313912
loss 2.298081
loss 2.282833
loss 2.278455
loss 2.255876
loss 2.227625
loss 2.240412
loss 2.203619
loss 2.211767
loss 2.174260
Test Loss: 
 Accuracy: 33.2%, Avg loss: 2.172681 

Epoch 2
-------------------------------
loss 2.179456
loss 2.170921
loss 2.125327
loss 2.150841
loss 2.089914
loss 2.036754
loss 2.068535
loss 1.990797
loss 2.002300
loss 1.929031
Test Loss: 
 Accuracy: 53.8%, Avg loss: 1.928287 

Epoch 3
-------------------------------
loss 1.955051
loss 1.926798
loss 1.821610
loss 1.870794
loss 1.749757
loss 1.700849
loss 1.727181
loss 1.621316
loss 1.643120
loss 1.536563
Test Loss: 
 Accuracy: 60.4%, Avg loss: 1.552305 

Epoch 4
-------------------------------
loss 1.617959
loss 1.577403
loss 1.430790
loss 1.510266
loss 1.381901
loss 1.377535
loss 1.395194
loss 1.311879
loss 1.346512
loss 1.243982
Test Loss: 
 Accuracy: 62.7%, Avg loss: 1.270038 

Epoch 5
-------------------------------
loss 1.352180
loss 1.326064
loss 1.1

これで最適化が完了です．では，早速試してみましょう．

In [17]:
import random

idx=random.randint(0,64) #0~64のランダムな値

model.eval() #モデルを検証モードで呼び出し．

with torch.no_grad():
        for X, y in test_dataloader:
            pred = model(X[idx])
            print("model\'s answer is :",pred.argmax(1)) #最大値をモデルの出したこたえとする
            print("The answer is :",y[idx]) #実際の答え
            break

model's answer is : tensor([5])
The answer is : tensor(5)


正確性が60%後半なのであやしいですが，しっかり予測できていますね．今回で，pytorchの基本的な機能の説明は終了となります。次回からはいよいよ使用事例の多いモデルについて学習していきます．お疲れ様でした．